In [0]:
import time
import math
import os
import os.path
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
import io
import requests


In [0]:
# os.chdir('C:\\Users\\Adam\\Desktop') #Go to where data is stored
# os.getcwd()

In [3]:
df=pd.read_csv('assistments.csv')
df

,problemlog_id,assignment_id,sequence_id,user_id,ARRS,assistment_id,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,first_response_time,overlap_time,cluster,pretest,posttest,avg_initial_three_correctness,avg_initial_#problems_to_master,avg_first_reassess
0,40790855,297282,5959,75063,0,34310,53621,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211
1,40790864,297282,5959,75063,0,34289,53537,2,2,1,1,0,1,4820,4820,0,0,1,0.671795,6.323529,0.684211
2,40790866,297282,5959,75063,0,34295,53561,3,3,1,1,0,1,8340,8340,0,0,1,0.671795,6.323529,0.684211
3,40790867,297282,5959,75063,0,34338,53733,4,4,1,1,0,1,10758,10758,0,0,1,0.671795,6.323529,0.684211
4,40897310,299510,5959,75063,0,34329,53697,5,1,2,1,0,0,10,10,1,0,1,0.671795,6.323529,0.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25400,41038106,301286,15675,93241,1,121297,231946,1,1,1,1,0,1,44759,44759,0,1,0,0.630769,5.470588,0.666667
25401,41038288,301286,15675,93241,1,121263,231911,2,2,1,0,0,5,55244,70970,0,1,0,0.630769,5.470588,0.666667
25402,41038533,301286,15675,93241,1,121220,231868,3,3,1,1,0,1,20336,20336,0,1,0,0.630769,5.470588,0.666667
25403,41038618,301286,15675,93241,1,121261,231909,4,4,1,1,0,1,12483,12483,0,1,0,0.630769,5.470588,0.666667


In [6]:
url = "https://drive.google.com/uc?export=download&id=0B3f_gAH-MpBmUmNJQ3RycGpJM0k"
s = requests.get(url).content
skill = pd.read_csv(io.StringIO(s.decode('latin')))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
skill.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [19]:
#checking that all problem_id in df is in skill


# df_problem = df['problem_id'].unique()
# skill_problem = skill['problem_id'].unique()
# df_problem.isin(skill_problem)


joined = df.join(skill, on='problem_id', rsuffix='skill!!!!')
sum(joined['school_id'].isnull())

#says 0 so i think it is ok!

0

In [0]:
print(len(df['assistment_id'].unique()))
print(len(df['problem_id'].unique()))
print(len(df['sequence_id'].unique())) #short. order appears?
print(len(df['assignment_id'].unique())) #as long as the DF. Seems like unique
# print(len(df['problemlog_id'].unique())) : 25405

3165
3165
33
817


In [0]:
df[df['problem_id'] == df['problem_id'][8]]
df = df.reset_index()
# df['problem_id'][0]

In [0]:
df['problem_id'] == df['assistment_id']

0        False
1        False
2        False
3        False
4        False
         ...  
25400    False
25401    False
25402    False
25403    False
25404    False
Length: 25405, dtype: bool

In [0]:
#assuming sequence_id is the skill_id
count = df.groupby('sequence_id').count()[['user_id']]
count = count.reset_index()
count = count.rename(columns={"user_id": "count_of_skill_tested"})
count #count of how much each sequence has been used

,sequence_id,count_of_skill_tested
0,5959,482
1,5962,377
2,6018,1272
3,6057,748
4,6848,532
5,6851,692
6,6854,564
7,6876,489
8,6895,345
9,6915,252


In [0]:
new_df = df.join(count.set_index('sequence_id'), on='sequence_id') #has the count of each sequence appearing as column
new_df.head()

,index,problemlog_id,assignment_id,sequence_id,user_id,ARRS,assistment_id,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,first_response_time,overlap_time,cluster,pretest,posttest,avg_initial_three_correctness,avg_initial_#problems_to_master,avg_first_reassess,count_of_skill_tested
0,0,40790855,297282,5959,75063,0,34310,53621,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,482
1,1,40790864,297282,5959,75063,0,34289,53537,2,2,1,1,0,1,4820,4820,0,0,1,0.671795,6.323529,0.684211,482
2,2,40790866,297282,5959,75063,0,34295,53561,3,3,1,1,0,1,8340,8340,0,0,1,0.671795,6.323529,0.684211,482
3,3,40790867,297282,5959,75063,0,34338,53733,4,4,1,1,0,1,10758,10758,0,0,1,0.671795,6.323529,0.684211,482
4,4,40897310,299510,5959,75063,0,34329,53697,5,1,2,1,0,0,10,10,1,0,1,0.671795,6.323529,0.684211,482


In [0]:
#looking at how many problems per skill
#we can look at which problems tended to result in higher post test score

# sp[sp['sequence_id'] == 5959]
# sp[sp['problem_id'] == 53621]
sp = new_df.groupby(['sequence_id', 'problem_id']).count()[['index']]
sp = sp.rename(columns={'index':'count'})
sp.sort_values('count')

count
sequence_id problem_id       
6854        85358           1
6915        86948           1
            86947           1
            86938           1
10195       161864          1
...                       ...
6018        61106          49
            61102          49
            61112          50
            61097          50
            61092          50

[3165 rows x 1 columns]

In [0]:
#Basic renaming


In [0]:
for col in df.columns:
    print(col)
    print (len(df[col].value_counts()))

problemlog_id
25405
assignment_id
817
sequence_id
33
user_id
99
ARRS
2
assistment_id
3165
problem_id
3165
sequence_opp
109
assignment_opp
91
ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat)
10
correct
2
hint_count
6
attempt_count
51
first_response_time
17891
overlap_time
18877
cluster
7
pretest
2
posttest
2
avg_initial_three_correctness
74
avg_initial_#problems_to_master
76
avg_first_reassess
38


In [0]:
df.rename(columns={'avg_initial_problems_to_master': 'Numprobmas', 'avg_initial_three_correctness': 'first_three'}, inplace=True)
drop_list=['problemlog_id', 'first_response_time', 'overlap_time' , 'assistment_id' ]
df.drop(drop_list, axis=1, inplace=True)

In [0]:
#Get the users mean pretest scores to rank them by ability
pretest=df.groupby('user_id')['pretest'].mean().to_dict()
#Returns a dictionary sorted from lowest value to highest value
sort_pretest={k: v for k, v in sorted(pretest.items(), key=lambda item: item[1])}

#Return dict of students from 1-99 where loweest ability is 1 and highest is 99 as judged by pretest
sort_dict={}
for i in range (len(sort_pretest)):
    sort_dict[i+1]=list(sort_pretest.values())[i]
sort_dict

{1: 0.1572700296735905,
 2: 0.19362745098039216,
 3: 0.20474777448071216,
 4: 0.2140468227424749,
 5: 0.21676300578034682,
 6: 0.27184466019417475,
 7: 0.2727272727272727,
 8: 0.28,
 9: 0.29156626506024097,
 10: 0.2934472934472934,
 11: 0.30120481927710846,
 12: 0.3057142857142857,
 13: 0.3110236220472441,
 14: 0.33264033264033266,
 15: 0.34105960264900664,
 16: 0.34306569343065696,
 17: 0.3440366972477064,
 18: 0.3602941176470588,
 19: 0.36585365853658536,
 20: 0.371900826446281,
 21: 0.372972972972973,
 22: 0.3732394366197183,
 23: 0.3794466403162055,
 24: 0.3835616438356164,
 25: 0.38596491228070173,
 26: 0.3873626373626374,
 27: 0.40425531914893614,
 28: 0.4055727554179567,
 29: 0.40703517587939697,
 30: 0.4153846153846154,
 31: 0.42543859649122806,
 32: 0.42857142857142855,
 33: 0.4540229885057471,
 34: 0.4732510288065844,
 35: 0.4763636363636364,
 36: 0.4767932489451477,
 37: 0.48292682926829267,
 38: 0.4931506849315068,
 39: 0.5041666666666667,
 40: 0.5048076923076923,
 41: 0.51

In [0]:
#Replaces the user id by the rankings of pretest scores. 
df['user_id'].replace(list(sort_pretest.keys()), list(sort_dict.keys()), inplace=True)
df.head()

,assignment_id,sequence_id,user_id,ARRS,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,cluster,pretest,posttest,first_three,avg_initial_#problems_to_master,avg_first_reassess
0,297282,5959,41,0,53621,1,1,1,0,0,1,0,0,1,0.671795,6.323529,0.684211
1,297282,5959,41,0,53537,2,2,1,1,0,1,0,0,1,0.671795,6.323529,0.684211
2,297282,5959,41,0,53561,3,3,1,1,0,1,0,0,1,0.671795,6.323529,0.684211
3,297282,5959,41,0,53733,4,4,1,1,0,1,0,0,1,0.671795,6.323529,0.684211
4,299510,5959,41,0,53697,5,1,2,1,0,0,1,0,1,0.671795,6.323529,0.684211


In [0]:
df['ARRS'].value_counts()

0    16500
1     8905
Name: ARRS, dtype: int64

In [0]:
problem=df.groupby('problem_id')['pretest'].mean().to_dict()
#Returns a dictionary sorted from lowest value to highest value
sort_problem={k: v for k, v in sorted(problem.items(), key=lambda item: item[1])}

#Return dict of students from 1-99 where loweest ability is 1 and highest is 99 as judged by pretest
prob_dict={}
for i in range (len(sort_problem)):
    prob_dict[i+1]=list(sort_problem.values())[i]
prob_dict



{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 100: 0.0,
 101: 0.

In [0]:
df.groupby('problem_id')['ARRS'].sum()
#Returns a dictionary sorted from lowest value to highest value

problem_id
8462      0
8463      1
42831     2
45479     4
53533     1
         ..
231954    1
231955    3
231956    2
231958    3
231959    5
Name: ARRS, Length: 3165, dtype: int64

In [0]:
user_df=df.groupby('user_id')

In [0]:
df['post_id']=df['user_id']

In [0]:
posttest=df.groupby('user_id')['posttest'].mean().to_dict()
#Returns a dictionary sorted from lowest value to highest value
sort_posttest={k: v for k, v in sorted(posttest.items(), key=lambda item: item[1])}

#Return dict of students from 1-99 where loweest ability is 1 and highest is 99 as judged by pretest
sort_dict={}
for i in range (len(sort_posttest)):
    sort_dict[i+1]=list(sort_posttest.values())[i]
sort_dict

df['post_id'].replace(list(sort_posttest.keys()), list(sort_dict.keys()), inplace=True)
df.head()

,assignment_id,sequence_id,user_id,ARRS,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,cluster,pretest,posttest,first_three,avg_initial_#problems_to_master,avg_first_reassess,post_id
0,297282,5959,41,0,53621,1,1,1,0,0,1,0,0,1,0.671795,6.323529,0.684211,15
1,297282,5959,41,0,53537,2,2,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
2,297282,5959,41,0,53561,3,3,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
3,297282,5959,41,0,53733,4,4,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
4,299510,5959,41,0,53697,5,1,2,1,0,0,1,0,1,0.671795,6.323529,0.684211,15


In [0]:
posttest=df.groupby('sequence_id')['pretest'].mean().to_dict()
#Returns a dictionary sorted from lowest value to highest value
sort_posttest={k: v for k, v in sorted(posttest.items(), key=lambda item: item[1])}

#Return dict of students from 1-99 where loweest ability is 1 and highest is 99 as judged by pretest
sort_dict={}
for i in range (len(sort_posttest)):
    sort_dict[i+1]=list(sort_posttest.values())[i]
sort_dict

df['sequence_id'].replace(list(sort_posttest.keys()), list(sort_dict.keys()), inplace=True)
df.head()

,assignment_id,sequence_id,user_id,ARRS,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,cluster,pretest,posttest,first_three,avg_initial_#problems_to_master,avg_first_reassess,post_id
0,297282,29,41,0,53621,1,1,1,0,0,1,0,0,1,0.671795,6.323529,0.684211,15
1,297282,29,41,0,53537,2,2,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
2,297282,29,41,0,53561,3,3,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
3,297282,29,41,0,53733,4,4,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15
4,299510,29,41,0,53697,5,1,2,1,0,0,1,0,1,0.671795,6.323529,0.684211,15


In [0]:
sort_dict

{1: 0.19948186528497408,
 2: 0.19970193740685543,
 3: 0.22478991596638656,
 4: 0.25460829493087556,
 5: 0.30760233918128654,
 6: 0.31266846361185985,
 7: 0.3314553990610329,
 8: 0.3358695652173913,
 9: 0.35697940503432496,
 10: 0.3778801843317972,
 11: 0.41147439851943246,
 12: 0.4161849710982659,
 13: 0.4231678486997636,
 14: 0.4699537750385208,
 15: 0.49,
 16: 0.4956521739130435,
 17: 0.5241057542768274,
 18: 0.5771513353115727,
 19: 0.6069587628865979,
 20: 0.6253731343283582,
 21: 0.6831550802139037,
 22: 0.7023809523809523,
 23: 0.7145390070921985,
 24: 0.7191780821917808,
 25: 0.7193396226415094,
 26: 0.7413632119514473,
 27: 0.7718832891246684,
 28: 0.8026315789473685,
 29: 0.8153526970954357,
 30: 0.8351822503961965,
 31: 0.8604651162790697,
 32: 0.8699360341151386,
 33: 0.9325153374233128}

In [0]:
df['skill_post']=df['sequence_id']

In [0]:
df['problem_id'].value_counts()

61097     50
61112     50
61092     50
61106     49
61102     49
          ..
86269      1
161767     1
86938      1
87005      1
86252      1
Name: problem_id, Length: 3165, dtype: int64

In [0]:
posttest=df.groupby('sequence_id')['pretest'].mean().to_dict()
#Returns a dictionary sorted from lowest value to highest value
sort_posttest={k: v for k, v in sorted(posttest.items(), key=lambda item: item[1])}

#Return dict of students from 1-99 where loweest ability is 1 and highest is 99 as judged by pretest
sort_dict={}
for i in range (len(sort_posttest)):
    sort_dict[i+1]=list(sort_posttest.values())[i]
sort_dict

df['sequence_id'].replace(list(sort_posttest.keys()), list(sort_dict.keys()), inplace=True)
df.head()

,assignment_id,sequence_id,user_id,ARRS,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,cluster,pretest,posttest,first_three,avg_initial_#problems_to_master,avg_first_reassess,post_id,skill_post
0,297282,29,41,0,53621,1,1,1,0,0,1,0,0,1,0.671795,6.323529,0.684211,15,29
1,297282,29,41,0,53537,2,2,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15,29
2,297282,29,41,0,53561,3,3,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15,29
3,297282,29,41,0,53733,4,4,1,1,0,1,0,0,1,0.671795,6.323529,0.684211,15,29
4,299510,29,41,0,53697,5,1,2,1,0,0,1,0,1,0.671795,6.323529,0.684211,15,29
